# Scrape Weather Observations From Metconnect 'Observations by Day - White Island' Web Page

1. This notebook uses the html source of the web page.
2. Save the page as a name like '201810.html', for a file for October 2018.
3. Edit the code line that starts *htmlfile =*.
4. Run this notebook by clicking Cell -> Run All.
5. A CSV file named like '201810.csv' is saved.

In [194]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [195]:
from bs4 import BeautifulSoup
import re
import pandas as pd
from pandas.compat import StringIO
import numpy as np
import os

In [196]:
def cleanhtml(raw_html):
    #drop all html tags
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, ',', raw_html)
    return cleantext

**input html file**

In [197]:
htmlfile = '201810.html'

**csv filename for output**

In [198]:
name, ext = os.path.splitext(htmlfile)
csvfile = name+'.csv'

**parse html file**

In [199]:
raw_html = open(htmlfile).read()
soup = BeautifulSoup(raw_html, 'html.parser')

**observation dates**

In [201]:
s = pd.Series()

for dateblock in (soup.select("a[href*='daily?date=']")):
    d = pd.Series(str(dateblock).split('=')[2][:8])
    s = s.append(d)
s = pd.to_datetime(s)
s = s.reset_index(drop=True)

**weather observations**

In [202]:
#select the table (which contains the data we need)
bigtab = soup.select('table')
tbody = bigtab[0].tbody

In [ ]:
#dataframe for the data
df  = pd.DataFrame()

#extract rows of the data table, strip out html tags, add a comma separator, append each rows to the dataframe
for block in (tbody.find_all('tr')):
    obsdata = cleanhtml(str(block))
    d = pd.read_csv(StringIO(obsdata), sep=',', header=None)
    df = df.append(d, ignore_index=True)

#drop any rows that are all NaN, these are empty rows included by the klutzy data extraction procedure
df.dropna(axis='columns', how='all', inplace=True)

#assign column names,including dummy columns
columns = ['Date', 'Rainfall', 'Wind_dir', 'Wind_mean', 'Wind_gust', 'AirTemp_min', 'AirTemp_max', 'AirTemp_mean', '20', '22', '24', '26', '28' ]
df.columns = columns

#drop columns not required
df.drop(labels=['Date','20','22','24','26','28'], axis=1, inplace=True)

#replace any - symbols with NaN
df.replace('-',np.nan, inplace=True)

**merge obsevation dates into weather data**

In [203]:
df['Date'] = s
df.set_index('Date', inplace=True)

**convert string data to numeric**

In [204]:
df.Rainfall = pd.to_numeric(df.Rainfall)
df.Wind_mean = pd.to_numeric(df.Wind_mean)
df.AirTemp_min = pd.to_numeric(df.AirTemp_min)
df.AirTemp_max = pd.to_numeric(df.AirTemp_max)
df.AirTemp_mean = pd.to_numeric(df.AirTemp_mean)
df.Wind_gust = pd.to_numeric(df.Wind_gust)

**save data as CSV file**

In [206]:
df.to_csv(csvfile)